In [91]:
import pandas as pd
from nltk.tokenize import word_tokenize
import string
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
import numpy as np
from tensorflow.keras.layers import Dropout

In [69]:
df = pd.read_csv('fake-news/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [70]:
## TODO - Add preprocess to pd.DataFrame
def preprocess(df: pd.DataFrame, column_name: str) -> None:
    df[column_name] = df[column_name].apply(lambda x: str(x).lower())
    translation = str.maketrans('','', string.punctuation)
    stemmer = PorterStemmer()
    df[column_name] = df[column_name].apply(lambda sentence: ' '.join([stemmer.stem(word.translate(translation)) \
                                                                       for word in word_tokenize(sentence) \
                                                                       if word.isalpha() and word not in stopwords.words('english')]))
    return(None)
preprocess(df, 'title')

In [71]:
df.head()

,id,title,author,text,label
0,0,hous dem aid even see comey letter jason chaff...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,flynn hillari clinton big woman campu breitbart,Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,truth might get fire,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,civilian kill singl us airstrik identifi,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,iranian woman jail fiction unpublish stori wom...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [72]:
df.drop('author', axis = 1, inplace = True)
df.dropna(inplace = True)

In [73]:
X = df.drop('label', axis = 1)
y = df['label']

In [74]:
X.shape

(20761, 3)

In [75]:
y.shape

(20761,)

In [76]:
tf.__version__

'2.3.1'

#### One-Hot Representation

In [77]:
voc_size = 5000

In [78]:
df.head()

,id,title,text,label
0,0,hous dem aid even see comey letter jason chaff...,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,flynn hillari clinton big woman campu breitbart,Ever get the feeling your life circles the rou...,0
2,2,truth might get fire,"Why the Truth Might Get You Fired October 29, ...",1
3,3,civilian kill singl us airstrik identifi,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,iranian woman jail fiction unpublish stori wom...,Print \nAn Iranian woman has been sentenced to...,1


In [79]:
onehot_repr = [one_hot(sentence, voc_size) for sentence in df.title]
onehot_repr[:5]

[[4136, 441, 4488, 1428, 966, 4385, 1793, 3201, 2630, 2740],
 [681, 1034, 3368, 2575, 948, 3569, 2876],
 [1415, 895, 2422, 4421],
 [4649, 3472, 1337, 1017, 896, 2088],
 [4827, 948, 302, 2082, 4354, 4631, 948, 1552, 3036, 1041]]

#### Embedding Representation

In [80]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding = 'pre', maxlen = sent_length)
print(embedded_docs)

[[   0    0    0 ... 3201 2630 2740]
 [   0    0    0 ...  948 3569 2876]
 [   0    0    0 ...  895 2422 4421]
 ...
 [   0    0    0 ... 1078 2525 2459]
 [   0    0    0 ... 1844  258 4375]
 [   0    0    0 ...    0 2581 1906]]


In [81]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4136,
        441, 4488, 1428,  966, 4385, 1793, 3201, 2630, 2740], dtype=int32)

In [82]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [84]:
np.array(embedded_docs).shape, y.shape

((20761, 20), (20761,))

In [85]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [86]:
X_final[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4136,
        441, 4488, 1428,  966, 4385, 1793, 3201, 2630, 2740], dtype=int32)

In [87]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

#### Model Training

In [89]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 64)

Epoch 1/10
218/218 [==============================] - 5s 24ms/step - loss: 0.3152 - accuracy: 0.8590 - val_loss: 0.1987 - val_accuracy: 0.9174
Epoch 2/10
218/218 [==============================] - 5s 21ms/step - loss: 0.1441 - accuracy: 0.9434 - val_loss: 0.1886 - val_accuracy: 0.9213
Epoch 3/10
218/218 [==============================] - 6s 25ms/step - loss: 0.1018 - accuracy: 0.9643 - val_loss: 0.2061 - val_accuracy: 0.9257
Epoch 4/10
218/218 [==============================] - 5s 22ms/step - loss: 0.0756 - accuracy: 0.9728 - val_loss: 0.2302 - val_accuracy: 0.9278
Epoch 5/10
218/218 [==============================] - 5s 24ms/step - loss: 0.0475 - accuracy: 0.9841 - val_loss: 0.2382 - val_accuracy: 0.9190
Epoch 6/10
218/218 [==============================] - 5s 22ms/step - loss: 0.0289 - accuracy: 0.9907 - val_loss: 0.3559 - val_accuracy: 0.9210
Epoch 7/10
218/218 [==============================] - 5s 22ms/step - loss: 0.0168 - accuracy: 0.9947 - val_loss: 0.4322 - val_accuracy: 0.9191

#### Adding Dropout

In [92]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [93]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 64)

Epoch 1/10
218/218 [==============================] - 5s 23ms/step - loss: 0.3304 - accuracy: 0.8442 - val_loss: 0.1857 - val_accuracy: 0.9208
Epoch 2/10
218/218 [==============================] - 5s 22ms/step - loss: 0.1503 - accuracy: 0.9408 - val_loss: 0.1777 - val_accuracy: 0.9263
Epoch 3/10
218/218 [==============================] - 5s 22ms/step - loss: 0.1142 - accuracy: 0.9579 - val_loss: 0.1843 - val_accuracy: 0.9254
Epoch 4/10
218/218 [==============================] - 5s 22ms/step - loss: 0.0844 - accuracy: 0.9694 - val_loss: 0.2199 - val_accuracy: 0.9232
Epoch 5/10
218/218 [==============================] - 5s 22ms/step - loss: 0.0633 - accuracy: 0.9786 - val_loss: 0.2111 - val_accuracy: 0.9229
Epoch 6/10
218/218 [==============================] - 5s 24ms/step - loss: 0.0473 - accuracy: 0.9842 - val_loss: 0.2925 - val_accuracy: 0.9175
Epoch 7/10
218/218 [==============================] - 5s 23ms/step - loss: 0.0360 - accuracy: 0.9872 - val_loss: 0.2778 - val_accuracy: 0.9162

#### Performance Metrics and Accuracy

In [94]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
y_pred = model.predict_classes(X_test)
confusion_matrix(y_test, y_pred)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([[3110,  324],
       [ 258, 3160]])

In [95]:
accuracy_score(y_test, y_pred)

0.9150612959719789